In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

In [2]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200806


import datetime 
base = datetime.date(2020,2,1)
for i in range(0,10): 
    d=base + datetime.timedelta(i)
    d=d.strftime("%Y%m%d")
    print(f"N029kakutokuruiseki{d}.csv")

from datetime import datetime, timedelta

def date_range(start_date: datetime, end_date: datetime):
    diff = (end_date - start_date).days + 1
    return (start_date + timedelta(i) for i in range(diff))

In [3]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

ディレクトリ

In [4]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")

In [5]:
#承認月
approval_month = "5月"
approval_yymmdd = 20200531

#Input_file
#①承認リスト（当月分）
input_file_1 = "【C103】N029（2020年5月入会利用実績.xlsx"

#②承認リスト（前月分）
input_file_2 = "N029（2020年5月度入会）.xlsx"

#③ デジガレ確認用リスト
input_file_3 = "20200721〆【JFRカード御中】成果データ&問い合わせ(ユーザー問い合わせ含).xlsx"


#Output_file
#①オンライン承認リスト
output_file_1 = f"online_利用条件_{today_yyyymmdd}" 
print("Output①⇒",output_file_1)

#②アフィリエイト月次成果承認確定リスト
output_file_2 = f"確認済み成果データ_利用条件_{today_yyyymmdd}" 
print("Output②⇒",output_file_2)


Output①⇒ online_利用条件_20200806
Output②⇒ 確認済み成果データ_利用条件_20200806


①承認リスト（利用実績1カ月分)

In [10]:
print(input_file_1)

df_app = pd.read_excel(data_dir /r"input"/f"{input_file_1}")
df_app.head()

【C103】N029（2020年5月入会利用実績.xlsx


,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD,カード売上額合計
0,20200501,1,0,D02,1,2,1,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602331559,30,12.0,2000,NaN,2.000010e+09,2,0,0
1,20200501,2,3,D02,1,2,1,0,3,1,1,0,NaN,NaN,NaN,NaN,155051921869919,63,12.0,10000,NaN,1.000001e+10,2,0,0
2,20200501,5,0,D00,2,31,1,1,3,104,2,0,D00,1.0,31.0,1.0,155603602532610,56,47.0,4000,NaN,4.000000e+03,1,0,0
3,20200501,4,0,D00,1,31,1,1,3,106,2,0,D00,1.0,2.0,1.0,155603602532644,31,12.0,3000,NaN,6.000000e+03,0,0,0
4,20200501,1,0,D00,4,31,2,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921869539,37,52.0,11000,40.0,1.100000e+04,2,1,6000


In [8]:
#df_app = df_app.rename(columns={'カード売上額合計':'2020年5月カード売上額合計'})

In [11]:
print(df_app.shape)
print(df_app.index)
print(df_app.columns)
print(df_app.dtypes)

(2324, 25)
RangeIndex(start=0, stop=2324, step=1)
Index(['カード加入日', '会員カード発行区分', '続柄', 'カード提携会社1CD', 'カード提携会社2CD', 'カード種類CD',
       'カードブランドCD', '切替CD', '勧誘CD', '受付部支店', '性別CD', '再発行CD', '旧カード提携会社1CD',
       '旧カード提携会社2CD', '旧カード種類CD', '旧カードブランドCD', '入会申込番号', '満年齢', 'リボ支払方法CD',
       '勧誘団体CD', '提携カード特殊エリア', 'エンボス内容', 'カード年会費CD', 'リボ利用対象CD', 'カード売上額合計'],
      dtype='object')
カード加入日           int64
会員カード発行区分       object
続柄               int64
カード提携会社1CD      object
カード提携会社2CD       int64
カード種類CD          int64
カードブランドCD        int64
切替CD             int64
勧誘CD             int64
受付部支店            int64
性別CD             int64
再発行CD            int64
旧カード提携会社1CD     object
旧カード提携会社2CD    float64
旧カード種類CD       float64
旧カードブランドCD     float64
入会申込番号           int64
満年齢              int64
リボ支払方法CD       float64
勧誘団体CD           int64
提携カード特殊エリア     float64
エンボス内容         float64
カード年会費CD         int64
リボ利用対象CD         int64
カード売上額合計         int64
dtype: object


オンライン入会客を特定

特定加入月&クレカ利用金額5000円以上に限定

In [12]:
df_app =df_app[(df_app["会員カード発行区分"] == "1") 
               & (df_app["続柄"] == 0) 
               & ~(df_app["カード提携会社2CD"] == 95) 
               & (df_app["切替CD"] == 0)
               & (df_app["カード加入日"] <= approval_yymmdd)
               & (df_app["カード売上額合計"] >= 5000)
               & (df_app["受付部支店"] == 299)]

In [13]:
#含まれてないか確認
print(df_app["会員カード発行区分"].unique())
print(df_app["続柄"].unique())
print(df_app["カード提携会社2CD"].unique())
print(df_app["切替CD"].unique())
print(df_app["受付部支店"].unique())
df_app.shape[0]

['1']
[0]
[ 4  1  8 18]
[0]
[299]


151

In [14]:
#"成果"カラムを追加
df_app['成果'] = "可"
df_app['参照用'] = df_app['入会申込番号'] #"参照用"はデジガレとのマッチング用

df_app=df_app[['入会申込番号','成果','カード加入日','参照用','カード売上額合計']]
df_app.head()

,入会申込番号,成果,カード加入日,参照用,カード売上額合計
4,155051921869539,可,20200501,155051921869539,6000
17,155051921869471,可,20200501,155051921869471,30000
40,155051921869893,可,20200501,155051921869893,21320
41,155051921869604,可,20200501,155051921869604,5333
100,155051921870354,可,20200501,155051921870354,6060


CSVアウトプット

In [15]:
df_tmp = df_app.drop(columns=['参照用'])
df_tmp.to_csv(data_dir /r"output"/f"{output_file_1}.csv", index=False,header=True,encoding='shift-jis')

② デジガレ確認用リスト ※利用条件シート

In [17]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 2)
df_dg.head()

20200721〆【JFRカード御中】成果データ&問い合わせ(ユーザー問い合わせ含).xlsx


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日
0,37.0,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,保留,保留
1,37.0,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,保留,保留
2,37.0,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,保留,保留
3,37.0,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000,保留,保留
4,5954.0,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000,保留,保留


In [18]:
print(bk.sheet_names[2])

成果データ（5月～利用条件追加媒体）


In [19]:
df_dg.shape[0]

684

In [20]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果','カード発行日'])
print(df_dg.dtypes)

ASID             float64
メディア名             object
媒体チャネル            object
クリック              object
申込        datetime64[ns]
参照用                int64
ASP名              object
単価                 int64
dtype: object


In [21]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(684, 8)
RangeIndex(start=0, stop=684, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '参照用', 'ASP名', '単価'], dtype='object')
ASID               int32
メディア名             object
媒体チャネル            object
クリック              object
申込        datetime64[ns]
参照用                int64
ASP名              object
単価                 int64
dtype: object


C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [22]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,2020年5月
1,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,2020年5月
2,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,2020年5月
3,37,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000,2020年5月
4,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000,2020年5月


In [23]:
df_app.head()

,入会申込番号,成果,カード加入日,参照用,カード売上額合計
4,155051921869539,可,20200501,155051921869539,6000
17,155051921869471,可,20200501,155051921869471,30000
40,155051921869893,可,20200501,155051921869893,21320
41,155051921869604,可,20200501,155051921869604,5333
100,155051921870354,可,20200501,155051921870354,6060


In [24]:
print(df_app.dtypes)

入会申込番号       int64
成果          object
カード加入日       int64
参照用          int64
カード売上額合計     int64
dtype: object


承認データ（利用条件）とデジガレリストをマージ

In [26]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日,カード売上額合計


In [27]:
df_dg2 = df_dg2.copy()

In [28]:
#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [29]:
df_dg2[df_dg2['カード加入日'] == 0].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日,カード売上額合計
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,2020年5月,NaN,0,NaN
1,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,2020年5月,NaN,0,NaN
2,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,2020年5月,NaN,0,NaN
3,37,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000,2020年5月,NaN,0,NaN
4,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000,2020年5月,NaN,0,NaN


In [30]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "保留"

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg2)):
    if df_dg2['カード加入日'][i] == 0:
        df_dg2.loc[:,'カード加入日'][i] = "保留"

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\crie072\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [32]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日,カード売上額合計
0,37,げん玉,ポイント,2020-05-01 00:06:29,2020-05-01 00:22:04,155051921869356,BGT,9000,2020年5月,保留,保留,NaN
1,37,げん玉,ポイント,2020-05-01 01:21:40,2020-05-01 01:28:04,155051921869380,BGT,9000,2020年5月,保留,保留,NaN
2,37,げん玉,ポイント,2020-05-01 06:27:16,2020-05-01 06:34:04,155051921869422,BGT,9000,2020年5月,保留,保留,NaN
3,37,げん玉,ポイント,2020-05-01 07:46:58,2020-05-01 08:02:04,155051921869489,BGT,9000,2020年5月,保留,保留,NaN
4,5954,GetMoney!(JIPC参加企業)（連携用）,ポイント,2020-05-01 11:33:44,2020-05-01 11:46:06,155051921869679,BGT,9000,2020年5月,保留,保留,NaN


集計

In [33]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

0

In [34]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,保留
申込年月,
2020年5月,94
2020年6月,183
2020年7月,407


In [34]:
#CSVアウトプット
df_dg2.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[2]}.csv", index=False,header=True,encoding='shift-jis')

③ デジガレ確認用リスト ※利用条件シート　先月の成果報告が正しくなかったため再度確認

In [43]:
print(input_file_3)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file_3}", sheet_name = 3)
df_dg.head()

20200721〆【JFRカード御中】成果データ&問い合わせ(ユーザー問い合わせ含).xlsx


,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,成果,カード発行日
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,可,20200615
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,可,20200611
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,可,20200525
3,21,ポイントインカム,ポイント,2020-05-01 07:24:26,2020-05-01 07:35:04,155051921869471,BGT,9000,可,20200501
4,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,可,20200525


In [36]:
print(bk.sheet_names[3])

成果データ（5月～利用条件追加媒体）_承認返却済み


In [37]:
df_dg.shape[0]

364

In [44]:
#余計なカラム削除
#df_dg = df_dg.drop(columns=['Unnamed: 9','成果'])
df_dg = df_dg.drop(columns=['成果','カード発行日'])
print(df_dg.dtypes)

ASID               int64
メディア名             object
媒体チャネル            object
クリック      datetime64[ns]
申込        datetime64[ns]
参照用                int64
ASP名              object
単価                 int64
dtype: object


In [45]:
#ASIDの整数化
for i in range(len(df_dg)):
    if pd.isnull(df_dg['ASID'][i]) == True:
        df_dg.loc[:,'ASID'][i] = 0
df_dg = df_dg.astype({'ASID':int})

print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(364, 8)
RangeIndex(start=0, stop=364, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '参照用', 'ASP名', '単価'], dtype='object')
ASID               int32
メディア名             object
媒体チャネル            object
クリック      datetime64[ns]
申込        datetime64[ns]
参照用                int64
ASP名              object
単価                 int64
dtype: object


In [46]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,2020年5月
3,21,ポイントインカム,ポイント,2020-05-01 07:24:26,2020-05-01 07:35:04,155051921869471,BGT,9000,2020年5月
4,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,2020年5月


In [47]:
df_app.head()

,入会申込番号,成果,カード加入日,参照用,カード売上額合計
4,155051921869539,可,20200501,155051921869539,6000
17,155051921869471,可,20200501,155051921869471,30000
40,155051921869893,可,20200501,155051921869893,21320
41,155051921869604,可,20200501,155051921869604,5333
100,155051921870354,可,20200501,155051921870354,6060


In [48]:
print(df_app.dtypes)

入会申込番号       int64
成果          object
カード加入日       int64
参照用          int64
カード売上額合計     int64
dtype: object


In [49]:
#承認データ（df_app）をマージする前に、df_app余計なカラムを削除
df_app_tmp = df_app.drop(columns=['入会申込番号'])

#承認データの"成果"をマージ
df_dg2 = pd.merge(df_dg,df_app_tmp, on =['参照用'], how = 'left')
df_dg2[df_dg2['成果'] == '可'].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日,カード売上額合計
3,21,ポイントインカム,ポイント,2020-05-01 07:24:26,2020-05-01 07:35:04,155051921869471,BGT,9000,2020年5月,可,20200501.0,30000.0
6,37,げん玉,ポイント,2020-05-01 08:59:59,2020-05-01 09:09:04,155051921869539,BGT,9000,2020年5月,可,20200501.0,6000.0
8,57,ライフメディア,ポイント,2020-05-01 10:36:23,2020-05-01 11:01:05,155051921869604,BGT,9000,2020年5月,可,20200501.0,5333.0
16,21,ポイントインカム,ポイント,2020-05-01 17:34:01,2020-05-01 17:47:05,155051921870354,BGT,9000,2020年5月,可,20200501.0,6060.0
19,57,ライフメディア,ポイント,2020-05-01 19:59:01,2020-05-01 20:20:05,155051921870495,BGT,9000,2020年5月,可,20200507.0,34821.0


In [50]:
df_dg2 = df_dg2.copy()

In [52]:
#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['カード加入日'][i]) == True:
        df_dg2.loc[:,'カード加入日'][i] = 0
        
df_dg2 = df_dg2.astype({'カード加入日':int})

In [53]:
df_dg2[df_dg2['カード加入日'] == 0].head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日,カード売上額合計
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月,NaN,0,NaN
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月,NaN,0,NaN
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,2020年5月,NaN,0,NaN
4,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,2020年5月,NaN,0,NaN
5,37,げん玉,ポイント,2020-05-01 07:59:02,2020-05-01 08:25:06,155051921869513,BGT,9000,2020年5月,NaN,0,NaN


In [55]:
#"成果"がNanのものは[保留]に置換する
for i in range(len(df_dg2)):
    if pd.isnull(df_dg2['成果'][i]) == True:
        df_dg2.loc[:,'成果'][i] = "保留"
        
#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg2)):
    if df_dg2['カード加入日'][i] == 0:
        df_dg2.loc[:,'カード加入日'][i] = "保留"

In [56]:
df_dg2.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード加入日,カード売上額合計
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月,保留,保留,NaN
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月,保留,保留,NaN
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,2020年5月,保留,保留,NaN
3,21,ポイントインカム,ポイント,2020-05-01 07:24:26,2020-05-01 07:35:04,155051921869471,BGT,9000,2020年5月,可,20200501,30000.0
4,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,2020年5月,保留,保留,NaN


集計

In [57]:
#客数
df_dg2[df_dg2['成果'] == '可'].shape[0]

114

In [58]:
pd.crosstab(df_dg2['申込年月'],df_dg2['成果'])

成果,保留,可
申込年月,,
2020年5月,196,114
2020年6月,54,0


カード利用条件ありにしかカード加入日が入力されていないので、再付与

In [67]:
print(input_file_1)

df_new = pd.read_excel(data_dir /r"input"/f"{input_file_1}",usecols =['カード加入日','入会申込番号'])
df_new = df_new.rename(columns={'入会申込番号':'参照用'}) #"参照用"はデジガレとのマッチング用
df_new.head()

【C103】N029（2020年5月入会利用実績.xlsx


,カード加入日,参照用
0,20200501,155603602331559
1,20200501,155051921869919
2,20200501,155603602532610
3,20200501,155603602532644
4,20200501,155051921869539


In [76]:
df_dg3 = df_dg2.drop(columns={'カード加入日'})
df_dg3 = pd.merge(df_dg3,df_new, on =['参照用'], how = 'left')

#マージしたらカード加入日がfloat型になってしまったため、整数型に戻す
for i in range(len(df_dg3)):
    if pd.isnull(df_dg3['カード加入日'][i]) == True:
        df_dg3.loc[:,'カード加入日'][i] = 0
        
df_dg3 = df_dg3.astype({'カード加入日':int})

       
#カード加入日が0のものは[保留]に置換する
for i in range(len(df_dg3)):
    if df_dg3['カード加入日'][i] == 0:
        df_dg3.loc[:,'カード加入日'][i] = "保留"

C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\crie072\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\crie072\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, 

In [77]:
df_dg3.head()

,ASID,メディア名,媒体チャネル,クリック,申込,参照用,ASP名,単価,申込年月,成果,カード売上額合計,カード加入日
0,37,げん玉,ポイント,2020-05-01 00:31:25,2020-05-01 00:52:05,155051921869372,BGT,9000,2020年5月,保留,NaN,保留
1,21,ポイントインカム,ポイント,2020-05-01 06:15:50,2020-05-01 06:25:05,155051921869414,BGT,9000,2020年5月,保留,NaN,保留
2,21,ポイントインカム,ポイント,2020-05-01 06:51:10,2020-05-01 07:03:03,155051921869455,BGT,9000,2020年5月,保留,NaN,20200525
3,21,ポイントインカム,ポイント,2020-05-01 07:24:26,2020-05-01 07:35:04,155051921869471,BGT,9000,2020年5月,可,30000.0,20200501
4,21,ポイントインカム,ポイント,2020-05-01 07:53:11,2020-05-01 08:06:04,155051921869497,BGT,9000,2020年5月,保留,NaN,20200525


In [78]:
#CSVアウトプット
df_dg3.to_csv(data_dir /r"output"/f"{output_file_2}_{bk.sheet_names[3]}.csv", index=False,header=True,encoding='shift-jis')